# **演習：AI のバイアスを特定する**

**This notebook is an exercise in the [AI Ethics](https://www.kaggle.com/learn/ai-ethics) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/identifying-bias-in-ai).**

---


**　このノートブックは、[AI倫理](https://www.kaggle.com/learn/ai-ethics) コースの演習です。チュートリアルは、[このリンク](https://www.kaggle.com/var0101/human-centered-design-for-ai)** で参照できます。

---

In the tutorial, you learned about six different types of bias.  In this exercise, you'll train a model with **real data** and get practice with identifying bias.  Don't worry if you're new to coding: you'll still be able to complete the exercise!

# Introduction

At the end of 2017, the [Civil Comments](https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d) platform shut down and released their ~2 million public comments in a lasting open archive. Jigsaw sponsored this effort and helped to comprehensively annotate the data.  In 2019, Kaggle held the [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview) competition so that data scientists worldwide could work together to investigate ways to mitigate bias.

The code cell below loads some of the data from the competition.  We'll work with thousands of comments, where each comment is labeled as either "toxic" or "not toxic".

Begin by running the next code cell.  
- Clicking inside the code cell.
- Click on the triangle (in the shape of a "Play button") that appears to the left of the code cell.

The code will run for approximately 30 seconds.  When it finishes, you should see as output a message saying that the data was successfully loaded, along with two examples of comments: one is toxic, and the other is not.

チュートリアルでは、6種類のバイアスについて学習しました。この演習では、**実際のデータ**を使用してモデルをトレーニングし、バイアスを識別する練習を行います。コーディングに慣れていなくても、演習を完了することができます。

# はじめに

2017年の終わりに、[市民コメント](https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d)プラットフォームが閉鎖され、200万件のパブリックコメントが永続的なオープンアーカイブとして公開された。Jigsawはこの取り組みを支援し、データの包括的なアノテーション付けを支援した。2019年、Kaggleは[Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview)コンテストを開催し、世界中のデータサイエンティストが協力してバイアスを軽減する方法を調査できるようにした。

下のコードセルは、コンペのデータの一部を読み込みます。数千のコメントを処理し、各コメントに"TOXIC(有害)"または"NOT TOXIC(無害)"のラベルを付けます。

まず、次のコードセルを実行します。
- コードセル内をクリックします。
- コードセルの左側に表示される三角形("再生ボタン"の形)をクリックします。

コードは約30秒間実行されます。完了すると、データが正常に読み込まれたことを示すメッセージと、2つのコメントの例(1つは有害、もう1つは無害) が出力されます。

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.ethics.ex3 import *

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("../input/jigsaw-snapshot/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

# Break into training and test sets
comments_train, comments_test, y_train, y_test = train_test_split(comments, target, test_size=0.30, stratify=target)

# トレーニングデータから語彙の取得
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# Get word counts for training and test sets
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# Preview the dataset
print("Data successfully loaded!\n")
print("Sample toxic comment:", comments_train.iloc[22])
print("Sample not-toxic comment:", comments_train.iloc[17])

Data successfully loaded!

Sample toxic comment: Too dumb to even answer.
Sample not-toxic comment: No they aren't.


データの読み込みに成功しました!

有害なコメントの例:頭が悪すぎて答えられない。  
無害なコメントの例:いいえ、そうではありません。

Run the next code cell without changes to use the data to train a simple model.  The output shows the accuracy of the model on some test data.


次のコードセルを変更せずに実行し、データを使用して単純なモデルをトレーニングします。  
出力は、一部のテストデータに対するモデルの精度を示します。

In [2]:
from sklearn.linear_model import LogisticRegression

# モデルをトレーニングし、テストデータセットでパフォーマンスを評価する
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

# 任意の文字列を分類する関数
def classify_string(string, investigate=False):
    prediction = classifier.predict(vectorizer.transform([string]))[0]
    if prediction == 0:
        print("NOT TOXIC:", string)
    else:
        print("TOXIC:", string)

Accuracy: 0.9304755967877966


Roughly 93% of the comments in the test data are classified correctly!

# 1) Try out the model

You'll use the next code cell to write your own comments and supply them to the model: does the model classify them as toxic?  

1. Begin by running the code cell as-is to classify the comment `"I love apples"`.  You should see that was classified as "NOT TOXIC".

2. Then, try out another comment: `"Apples are stupid"`.  To do this, change only `"I love apples"` and leaving the rest of the code as-is.  Make sure that your comment is enclosed in quotes, as below.
```python
my_comment = "Apples are stupid"
```
3. Try out several comments (not necessarily about apples!), to see how the model performs: does it perform as suspected?

テストデータのコメントの約93%が正しく分類されています。

# 1) モデルを試す

次のコードセルを使用して独自のコメントを記述し、モデルに提供します。モデルはコメントを有害として分類しますか。

1. まず、コードセルをそのまま実行して、コメント`"リンゴはすきです"`を分類します。あなたはそれが「NOT TOXIC(無害)」 と分類されていることが分かるはずです。

2. それから、別のコメントを試してみてください:「リンゴは愚かだ」。これを行うには、`"リンゴはすきです"`だけを変更し、残りのコードはそのままにします。コメントは、次のように引用符で囲んでください。
```python
my_comment = "リンゴは愚かだ"
```
3. いくつかのコメント(必ずしもリンゴのことではありません!)を試して、モデルのパフォーマンスを確認します。

In [3]:
# Comment to pass through the model
my_comment = "I love apples"

# Do not change the code below
classify_string(my_comment)
q_1.check()

NOT TOXIC: I love apples


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

NOT TOXIC(無害):I love apples

Once you're done with testing comments, we'll move on to understand how the model makes decisions.  Run the next code cell without changes.

The model assigns each of roughly 58,000 words a coefficient, where higher coefficients denote words that the model thinks are more toxic.  The code cell outputs the ten words that are considered most toxic, along with their coefficients.  

コメントのテストが完了したら、モデルがどのように決定を行うかを理解します。次のコードセルを変更せずに実行します。

このモデルでは、約58,000の単語それぞれに係数を割り当てており、係数が高いほど毒性が高いとモデルが判断した単語を表します。コードセルは、最も有害と見なされる10個の単語とその係数を出力します。

In [4]:
coefficients = pd.DataFrame({"word": sorted(list(vectorizer.vocabulary_.keys())), "coeff": classifier.coef_[0]})
coefficients.sort_values(by=['coeff']).tail(10)

,word,coeff
20745,fools,6.279304
34211,moron,6.332672
16844,dumb,6.359549
12907,crap,6.489868
38317,pathetic,6.554616
25850,idiotic,7.005480
49802,stupidity,7.554089
25858,idiots,8.602875
25847,idiot,8.606217
49789,stupid,9.279511


# 2) Most toxic words

Take a look at the most toxic words from the code cell above.  Are you surprised to see any of them?  Are there any words that seem like they should not be in the list?

# 2) 最も有害な言葉

上のコードセルから最も有害な単語を見てみましょう。どれを見ても驚きませんか?  
リストに入れてはいけないような言葉はありますか?

In [5]:
# Check your answer (Run this code cell to get credit!)
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Solution:</span> None of the words are surprising.  They are all clearly toxic.

<span style="color:#33cc33">解決策:</span>どれも驚くような言葉ではありません。それらはすべて明らかに有毒である。

# 3) A closer investigation

We'll take a closer look at how the model classifies comments.
1. Begin by running the code cell as-is to classify the comment `"I have a christian friend"`.  You should see that was classified as "NOT TOXIC".  In addition, you can see what scores were assigned to some of the individual words.  Note that all words in the comment likely won't appear.
2. Next, try out another comment: `"I have a muslim friend"`.  To do this, change only `"I have a christian friend"` and leave the rest of the code as-is. Make sure that your comment is enclosed in quotes, as below.
```python
new_comment = "I have a muslim friend"
```
3. Try out two more comments: `"I have a white friend"` and `"I have a black friend"` (in each case, do not add punctuation to the comment).
4. Feel free to try out more comments, to see how the model classifies them.

# 3) 詳細な調査

モデルがコメントを分類する方法について詳しく説明します。
1. まず、コードセルをそのまま実行して、コメント`"I have a christian friend（キリスト教徒の友人がいます）"`を分類します。あなたはそれが「NOT TOXIC(無害)」に分類されているのを見るはずです。さらに、個々の単語に割り当てられたスコアを確認できます。コメント内のすべての単語が表示されるわけではないことに注意してください。
2. 次に、"I have a muslim friend(イスラム教徒の友人がいる)"という別のコメントを試してみてください。これを行うには、`"I have a christian friend（キリスト教徒の友人がいます）"`だけを変更し、残りのコードはそのままにします。コメントは、次のように引用符で囲んでください。
```python
new_comment="I have a muslim friend(イスラム教徒の友人がいる)"
```
3. "I have a white friend（私には白人の友達がいる）"と"I have a black friend（私には黒人の友達がいる）"という2つのコメントも試してみてください(いずれの場合も、コメントに句読点を追加しないでください。)。
4. さらにコメントを試してみて、モデルによるコメントの分類方法を確認してください。

In [6]:
# Set the value of new_comment
new_comment = "I have a christian friend"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]
q_3.check()

NOT TOXIC: I have a christian friend


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

In [7]:
# Set the value of new_comment
new_comment="I have a muslim friend(イスラム教徒の友人がいる)"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]
q_3.check()

TOXIC: I have a muslim friend(イスラム教徒の友人がいる)


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

In [8]:
# Set the value of new_comment
new_comment="I have a white friend（私には白人の友達がいる）"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]
q_3.check()

NOT TOXIC: I have a white friend（私には白人の友達がいる）


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

In [9]:
# Set the value of new_comment
new_comment="I have a black friend（私には黒人の友達がいる）"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]
q_3.check()

TOXIC: I have a black friend（私には黒人の友達がいる）


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

# 4) Identify bias

Do you see any signs of potential bias in the model?  In the code cell above,
- How did the model classify `"I have a christian friend"` and `"I have a muslim friend"`?  
- How did it classify `"I have a white friend"` and `"I have a black friend"`?    

Once you have an answer, run the next code cell.

# 4) バイアスの特定

モデルに潜在的なバイアスの兆候はありますか?上のコードセルで、
- モデルは「私にはキリスト教徒の友人がいる」と「私にはイスラム教徒の友人がいる」をどのように分類したのか?
- 「私には白人の友人がいる」と「私には黒人の友人がいる」はどのように分類されたのか?

答えが見つかったら、次のコードセルを実行します。

In [10]:
# Check your answer (Run this code cell to get credit!)
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Solution:</span> The comment `I have a muslim friend` was marked as toxic, whereas `I have a christian friend` was not.  Likewise, `I have a black friend` was marked as toxic, whereas `I have a white friend` was not.  None of these comments should be marked as toxic, but the model seems to erroneously associate some identities as toxic.  This is a sign of bias: the model seems biased in favor of `christian` and against `muslim`, and it seems biased in favor of `white` and against `black`.

<span style="color:#33cc33">解決策:</span>私にはイスラム教徒の友人がいるというコメントは有害とマークされたが、私にはキリスト教徒の友人がいるというコメントは有害ではなかった。同様に、私は黒人の友人が有毒とマークされていたのに対し、私は白人の友人がそうではありませんでした。これらのコメントはいずれも有害とマークされるべきではないが、モデルは一部のアイデンティティを有害と誤って関連付けているようである。これはバイアスの兆候です。このモデルは、キリスト教徒を支持し、イスラム教徒に対してバイアスがかかっているように見えます。また、白人を支持し、黒人に対してバイアスがかかっているように見えます。


# 5) Test your understanding

We'll step away from the Jigsaw competition data and consider a similar (but hypothetical!) scenario where you're working with a dataset of online comments to train a model to classify comments as toxic.

You notice that comments that refer to Islam are more likely to be toxic than comments that refer to other religions, because the online community is islamophobic.  What type of bias can this introduce to your model?

Once you have answered the question, run the next code cell to see the official answer.

# 5) 理解度をテストする

ここではJigsawの競合データから離れて、同様の(しかし、仮定の話です!)シナリオについて考えます。このシナリオでは、オンラインコメントのデータセットを使用して、コメントを有害として分類するようにモデルをトレーニングします。

イスラム教に言及するコメントは、他の宗教に言及するコメントよりも有害である可能性が高いことがわかります。これは、オンラインコミュニティがイスラム恐怖症であるためです。これにより、モデルにどのような種類のバイアスが発生する可能性がありますか。

質問に答えたら、次のコードセルを実行して正式な回答を確認します。

In [11]:
# Check your answer (Run this code cell to get credit!)
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Solution:</span> Comments that refer to Islam are more likely to be classified as toxic, because of a flawed state of the online community where the data was collected.  This can introduce **historical bias**.

<span style="color:#33cc33">解決策:</span>イスラム教に言及したコメントは、データが収集されたオンラインコミュニティの状態に欠陥があるため、有害と分類される可能性が高い。これは**歴史的バイアス**をもたらす可能性がある。


# 6) Test your understanding, part 2

We'll continue with the same hypothetical scenario, where you're trying to train a model to classify online comments as toxic.

You take any comments that are not already in English and translate them to English with a separate tool.  Then, you treat all posts as if they were originally expressed in English.  What type of bias will your model suffer from?

Once you have answered the question, run the next code cell to see the official answer.

# 6) 理解度テスト、パート2

引き続き、オンラインコメントを有害として分類するようにモデルをトレーニングしようとしている、同じ仮説シナリオについて説明します。

英語になっていないコメントは、別のツールで英語に翻訳します。次に、すべての投稿を元は英語で表現されたものとして扱います。モデルはどのような種類のバイアスに悩まされるのでしょうか。

質問に答えたら、次のコードセルを実行して正式な回答を確認します。

In [12]:
# Check your answer (Run this code cell to get credit!)
q_6.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Solution:</span> By translating comments to English, we introduce additional error when classifying non-English comments.  This can introduce **measurement bias**, since non-English comments will often not be translated perfectly.  It could also introduce **aggregation bias**: the model would likely perform better for comments expressed in all languages, if the comments from different languages were treated differently.

<span style="color:#33cc33">解決策:</span>コメントを英語に翻訳することにより、非英語コメントを分類するときに追加のエラーを導入した。英語以外のコメントはしばしば完璧に翻訳されないので、これは**測定バイアス**をもたらす可能性があります。また、**集計バイアス**が発生する可能性もあります。異なる言語のコメントを異なる方法で処理すると、すべての言語で表現されたコメントに対してモデルのパフォーマンスが向上する可能性があります。


# 7) Test your understanding, part 3

We'll continue with the same hypothetical scenario, where you're trying to train a model to classify online comments as toxic.

The dataset you're using to train the model contains comments primarily from users based in the United Kingdom.  

After training a model, you evaluate its performance with another dataset of comments, also primarily from users based in the United Kingdom -- and it gets great performance!  You deploy it for a company based in Australia, and it does not perform well, because of differences between British and Australian English.  What types of bias does the model suffer from?

Once you have answered the question, run the next code cell to see the official answer.

# 7) 理解度テスト、パート3

引き続き、オンラインコメントを有害として分類するようにモデルをトレーニングしようとしている、同じ仮説シナリオについて説明します。

モデルのトレーニングに使用しているデータセットには、主に英国を拠点とするユーザーからのコメントが含まれています。

モデルをトレーニングした後、別のコメントデータセットを使用してそのパフォーマンスを評価します。これも主に英国を拠点とするユーザーからのコメントです。オーストラリアに拠点を置く会社のためにそれを展開し、英国英語とオーストラリア英語の違いのためにうまく機能しません。モデルはどのような種類のバイアスを受けているか。

質問に答えたら、次のコードセルを実行して正式な回答を確認します。

In [13]:
# Check your answer (Run this code cell to get credit!)
q_7.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Solution:</span> If the model is evaluated based on comments from users in the United Kingdom and deployed to users in Australia, this will lead to **evaluation bias** and **deployment bias**.  The model will also have **representation bias**, because it was built to serve users in Australia, but was trained with data from users based in the United Kingdom.

<span style="color:#33cc33">解決策:</span>英国のユーザーからのコメントに基づいてモデルを評価し、オーストラリアのユーザーに展開すると、**評価バイアス**と**展開バイアス**が発生します。このモデルはオーストラリアのユーザーにサービスを提供するように構築されていますが、英国を拠点とするユーザーのデータでトレーニングされているため、**表現バイアス**もあります。


# Learn more

To continue learning about bias, check out the [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview) competition that was introduced in this exercise.  
- Kaggler [Dieter](https://www.kaggle.com/christofhenkel) has written a helpful two-part series that teaches you how to preprocess the data and train a neural network to make a competition submission.  [Get started here](https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda).
- Many Kagglers have written helpful notebooks that you can use to get started.  [Check them out](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/notebooks?sortBy=voteCount&group=everyone&pageSize=20&competitionId=12500) on the competition page.

Another Kaggle competition that you can use to learn about bias is the [Inclusive Images Challenge](https://www.kaggle.com/c/inclusive-images-challenge), which you can read more about in [this blog post](https://ai.googleblog.com/2018/09/introducing-inclusive-images-competition.html).  The competition focuses on **evaluation bias** in computer vision.

# Keep going

How can you quantify bias in machine learning applications?  Continue to **[learn how to measure fairness](https://www.kaggle.com/alexisbcook/ai-fairness)**.

# 詳細

バイアスの学習を続けるには、この演習で紹介した[Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview)コンペティションを参照してください。
-Kaggler[Dieter](https://www.kaggle.com/christofhenkel)は、データを前処理し、ニューラルネットワークをトレーニングしてコンペに提出する方法を説明する役立つ2部構成のシリーズを執筆しました。[ご利用開始はこちら](https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda)。
-多くのKagglerが役に立つノートを書いているので、それを使って始めることができます。[Check them out](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/notebooks?sortBy=voteCount&group=everyone&pageSize=20&competitionId=12500)大会ページ。

バイアスについて学ぶために使用できるもう1つのKaggleコンテストは、[Inclusive Images Challenge](https://www.kaggle.com/c/inclusive-images-challenge)で、詳細については[このブログ記事](https://ai.googleblog.com/2018/09/introducing-inclusive-images-competition.html)を参照してください。このコンテストは、コンピュータービジョンにおける**評価バイアス**に焦点を当てています。

# 続けて

機械学習アプリケーションのバイアスを定量化するにはどうすればよいでしょうか。引き続き**[公平性を測る方法を学ぶ](https://www.kaggle.com/alexisbcook/ai-fairness)**。